# Calculate averaged target distances over lags 

Uses the outputs of 0-1_target_distance.ipynb

In [8]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import to_monthly

In [15]:
# Parameters
vname = 'sst'
grid = '2x2'
lat_slice = (-10, 10)
lon_slice = (120, 290)
data_dir = '../data/cesm2'
is_scaled = True         # whether to consider the difference in variability over months
lags = [0, -3, -6, -9]   # lags to average over

# Read original target distance
f = f'{data_dir}/target_distance.nc'
msd = xr.open_dataarray(f)

# Read data
da = xr.open_dataarray(f'{data_dir}/{vname}_anomaly_{grid}.nc')
da = da.sel(lat=slice(*lat_slice), lon=slice(*lon_slice))
da_month = to_monthly(da)

# Domain-averaged variance of each month
variance = da_month.var(dim=['ens', 'year']).mean(dim=['lat', 'lon'])

# Scale MSD so that each month contribution is equal
if is_scaled:
    msd = msd / variance

/home/ktoride/.conda/envs/torch/lib/python3.11/site-packages/xarray/core/nputils.py:177: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [10]:
# Stack
msd_stack = msd.stack(time=('year', 'month'))

# Find dates to calculate shadowing distance
dates = pd.DataFrame(list(msd_stack.time.data), columns=['year', 'month'])
dates['day'] = 1
dates = pd.to_datetime(dates).rename('time')
dates = dates[-lags[-1]:]

In [13]:
msd_dates = []
for d in dates:
    msd_lags = []
    for lag in lags:
        # Reference lagged time
        lag_d = d + pd.DateOffset(months=lag)

        # Library target time
        library_year = msd_stack.lyear.data + d.year - lag_d.year

        # Shift time
        ds_shift = msd_stack.sel(time=(lag_d.year, lag_d.month), drop=True
                                ).assign_coords(lyear=library_year)
        
        msd_lags.append(ds_shift)
        
    msd_lag_mean = xr.concat(msd_lags, pd.Index(lags, name='lag')
                             ).mean(dim='lag', skipna=False)
    msd_dates.append(msd_lag_mean)
msd_shadow = xr.concat(msd_dates, dates, join='inner')
msd_shadow = to_monthly(msd_shadow)

In [16]:
# Output file name
lags_str = '_'.join([str(lag) for lag in lags])
if is_scaled:
    outf = f'{f.split(".nc")[0]}_scaled_{lags_str}.nc'
else:
    outf = f'{f.split(".nc")[0]}_{lags_str}.nc'

# Save
encoding = {msd.name: {'dtype': 'float32'}}
msd_shadow.to_netcdf(outf, encoding=encoding)
print(outf)

../data/cesm2/target_distance_scaled_0_-3_-6_-9.nc


In [17]:
msd_shadow

<xarray.DataArray 'sst' (lens: 100, lyear: 136, ens: 100, year: 136, month: 12)>
array([[[[[       nan,        nan,        nan, ..., 0.        ,
           0.        , 0.        ],
          [       nan,        nan,        nan, ..., 1.8907123 ,
           2.2351358 , 2.3038504 ],
          [       nan,        nan,        nan, ..., 5.8257885 ,
           5.429137  , 4.8633904 ],
          ...,
          [       nan,        nan,        nan, ..., 0.70535403,
           0.61905104, 0.56227815],
          [       nan,        nan,        nan, ..., 1.0884011 ,
           1.0423014 , 1.0545881 ],
          [       nan,        nan,        nan, ..., 2.1470358 ,
           1.887466  , 1.6837373 ]],

         [[       nan,        nan,        nan, ..., 1.7980822 ,
           2.1412668 , 2.2526076 ],
          [       nan,        nan,        nan, ..., 4.089144  ,
           3.8415701 , 3.0309663 ],
          [       nan,        nan,        nan, ..., 2.3631604 ,
           2.083228  , 2.1188605 ],
...
          [2.7498229 , 3.217105  , 3.4810383 , ..., 3.5501785 ,
           3.1025434 , 2.6811948 ],
          [0.4818399 , 0.48391432, 0.4434411 , ..., 0.36988235,
           0.34526458, 0.40692735],
          [1.2035222 , 0.820338  , 0.8294999 , ..., 0.37673146,
           0.34428465, 0.40165874]],

         [[       nan,        nan,        nan, ..., 2.404387  ,
           2.6523776 , 2.5758328 ],
          [1.8192996 , 2.1320858 , 2.3632553 , ..., 2.421066  ,
           2.1337461 , 1.9882524 ],
          [0.3258827 , 0.33397353, 0.4788    , ..., 0.88523716,
           1.0343543 , 0.99322075],
          ...,
          [2.017622  , 1.5515964 , 1.3041326 , ..., 1.615999  ,
           1.829463  , 2.0680344 ],
          [2.2829728 , 2.5193686 , 2.4630568 , ..., 2.6692781 ,
           2.3632767 , 1.9514637 ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ]]]]], dtype=float32)
Coordinates:
  * year     (year) int64 1865 1866 1867 1868 1869 ... 1996 1997 1998 1999 2000
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * ens      (ens) int64 1 2 3 4 5 6 7 8 9 10 ... 91 92 93 94 95 96 97 98 99 100
  * lyear    (lyear) int64 1865 1866 1867 1868 1869 ... 1996 1997 1998 1999 2000
  * lens     (lens) int64 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100